Two sets of points will be created, one of start and end points of the lines at crossings, and another for the start and end points of each of the original lines. 

In [ ]:
# function to return coordinates from arc point object
def coordinates_of(point_object):
    x = round(point_object.X,5)
    y = round(point_object.Y,5)
    
    return x,y

#funtion to return coordinates of the point 10 meters from the start of a line:
def pt_x_meters_from(number_of_meters_from, start_or_end, of_line_geometry):
    if start_or_end == 'start':
         pt_x_meters_from = line_geometry.positionAlongLine(
             (float(number_of_meters_from) / float(of_line_geometry.length)), True).firstPoint
         coordinates_of_pt = coordinates_of(pt_ten_meters_from)
    elif start_or_end == 'end':
         pt_x_meters_from = line_geometry.positionAlongLine(
             (float(number_of_meters_from) / float(of_line_geometry.length)), True).firstPoint
         coordinates_of_pt= coordinates_of(pt_ten_meters_from)
    else:
        print("""%s must be string of either 'start' or 'end'""" % start_or_end)
    
    return coordinates_of_pt


def angle_between(first_point_coordinates, last_point_coordinates):
    x1, y1 = first_point_coordinates[0] * 1.0, first_point_coordinates[1] * 1.0
    x2, y2 = last_point_coordinates[0] * 1.0, last_point_coordinates[1] * 1.0

    polarcoords = cmath.polar(complex(x2 - x1, y2 - y1))
    if math.degrees(polarcoords[1]) <= 90:
        degrees = 90 - math.degrees(polarcoords[1])
    else:
        degrees = 450 - math.degrees(polarcoords[1])

    return round(degrees, 1)


In [ ]:
local_geodatabase = os.getcwd() + '\\working_files.gdb'
local_shapefiles_directory = os.getcwd() + '\\network_shapefiles'
ap.env.workspace = local_shapefiles_directory

network_shapefiles = ap.ListFeatureClasses()
ap.env.workspace = local_geodatabase

for each_network in network_shapefiles:
    network_name = ap.Describe(local_shapefiles_directory + '//' + each_network).baseName
    network_abbrev = network_name[:3]
    dissolved_to_line_feature_class = network_name + '_dissolved_to_line'
    
    add_fields = ['start_angle',
                  'end_angle',
                  ]

    for field in add_fields:
        try:
            ap.AddField_management('%s_dissolved_to_line' % network_name,
                                       '%s_%s' % (network_abbrev, field),
                                       'STRING',
                                       '',
                                       '',
                                       '255')
        except:
            print('field is probably already present')
    
    with ap.da.UpdateCursor('%s_dissolved_to_line' % network_name,
                        ['SHAPE@',
                         'start_angle', 
                         'end_angle']) as angle_cursor:
        
        for start_and_end_angles in angle_cursor:
            the_lines = start_and_end_angles[0]
            
            start_point = coordinates_of(the_lines.firstPoint)
            point_10meters_from_start = pt_x_meters_from(10,'start',the_lines)
            start_angle = angle_between(start_point, point_10meters_from_start)
            
            end_point = coordinates_of(the_lines.lastPoint)
            point_10meters_from_end =  pt_x_meters_from(10,'end',the_lines)
            end_angle = angle_between(end_point, point_10meters_from_end)
            
            start_and_end_angles[1:2] = start_angle, end_angle
            angle_cursor.updateRow(start_and_end_angles)